In [45]:
import requests
import bs4
from bs4 import BeautifulSoup
from urllib.parse import urljoin
import pandas as pd

In [58]:
url_inicial = 'http://www.gestionypoliticapublica.cide.edu/ojscide/index.php/gypp/issue/archive?issuesPage=2#issues'
url_root = 'http://www.gestionypoliticapublica.cide.edu/ojscide/index.php/gypp/issue/archive?issuesPage=2#issues'
r=requests.get(url_inicial)

In [59]:
soup = BeautifulSoup(r.text, 'html.parser')

In [60]:
box = soup.find('div', id='issues')
volumen=box.findAll('div', id='issue')
vol = [x.find('a').get('href')for x in volumen]
vol=[urljoin(url_root,i) for i in vol]
vol

['http://www.gestionypoliticapublica.cide.edu/ojscide/index.php/gypp/issue/view/15',
 'http://www.gestionypoliticapublica.cide.edu/ojscide/index.php/gypp/issue/view/12',
 'http://www.gestionypoliticapublica.cide.edu/ojscide/index.php/gypp/issue/view/11',
 'http://www.gestionypoliticapublica.cide.edu/ojscide/index.php/gypp/issue/view/13']

In [61]:
def get_url_items(sopa, url): 
    box = soup.find('div', id='issues')
    volumen=box.findAll('div', id='issue')
    vol = [x.find('a').get('href')for x in volumen]
    vol=[urljoin(url_root,i) for i in vol]
    return vol

In [62]:
links_items=[]
i=0
while i<1:
    i+=1
    print(f'Estoy en la pagina {url_inicial}')
    r_pag=requests.get(url_inicial)
    s_p=BeautifulSoup(r_pag.text,'html.parser')
    links=get_url_items(s_p, url_inicial)
    links_items.append(links)

Estoy en la pagina http://www.gestionypoliticapublica.cide.edu/ojscide/index.php/gypp/issue/archive?issuesPage=2#issues


In [63]:
list_scraper=[]
for i in links_items:
    for j in i:
        list_scraper.append(j)
len(list_scraper)

4

In [64]:
uno=list_scraper[0]
r_item=requests.get(uno)
s_item=BeautifulSoup(r_item.text, 'html.parser')

In [65]:
#funcion para iniciar el escraper de cada libro
def scraper_book(url):
    content_book={}
    r=requests.get(url)
    tire='Gestión y Política Pública'
    a='V. Ciencias Sociales'
    tem='Administración Pública y Estudio de las Organizaciones'
    url='http://www.gestionypoliticapublica.cide.edu/'
    s_item=BeautifulSoup(r.text,'html.parser')
    #titulo de revista
    try:
        titulo=tire
        content_book['Titulo de revista']=tire
    except AttributeError:
        content_book['Titulo de revista']=None
    #area
    try:
        area=a
        content_book['Área']=a
    except AttributeError:
        content_book['Área']=None
    #tematica
    try:
        tema=tem
        content_book['Temática']=tem
    except AttributeError:
        content_book['Temática']=None
    #titulo libro
    try:
        titu=s_item.find('td', class_='tocTitle').get_text(strip=True)
        content_book['Titulo del libro']=titu
    except AttributeError:
        content_book['Titulo del libro']=None
    #resumen
    try:
        resu=s_item.find('a', class_='cover').get_text(strip=True)
        content_book['Resumen']=resu
    except AttributeError:
        content_book['Resumen']=None
    #Link incial
    try:
        inicial=url
        content_book['Link inicial']=url
    except AttributeError:
        content_book['Link inicial']=None
    #link articulo
    try:
        link=s_item.find('a', class_='file').get('href')
        content_book['Link articulo']=urljoin(url_root, link)
    except AttributeError:
        content_book['Link articulo']=None
    return content_book

In [66]:
list_scraper=list_scraper[0:4]
datos_book=[]
for idx, i in enumerate(list_scraper):
    print(f'estas escrapeando la pag {idx}')
    datos_book.append(scraper_book(i))

estas escrapeando la pag 0
estas escrapeando la pag 1
estas escrapeando la pag 2
estas escrapeando la pag 3


In [67]:
df_catalogo=pd.DataFrame(datos_book)
df_catalogo

,Titulo de revista,Área,Temática,Titulo del libro,Resumen,Link inicial,Link articulo
0,Gestión y Política Pública,V. Ciencias Sociales,Administración Pública y Estudio de las Organi...,"Coherencia y políticas públicas. Metas, instru...",None,http://www.gestionypoliticapublica.cide.edu/,http://www.gestionypoliticapublica.cide.edu/oj...
1,Gestión y Política Pública,V. Ciencias Sociales,Administración Pública y Estudio de las Organi...,Soluciones al problema. La gestión de la segur...,None,http://www.gestionypoliticapublica.cide.edu/,http://www.gestionypoliticapublica.cide.edu/oj...
2,Gestión y Política Pública,V. Ciencias Sociales,Administración Pública y Estudio de las Organi...,Formación de capital social comunitario a part...,None,http://www.gestionypoliticapublica.cide.edu/,http://www.gestionypoliticapublica.cide.edu/oj...
3,Gestión y Política Pública,V. Ciencias Sociales,Administración Pública y Estudio de las Organi...,El estudio del deporte y las políticas públicas,None,http://www.gestionypoliticapublica.cide.edu/,http://www.gestionypoliticapublica.cide.edu/oj...


In [68]:
df_catalogo.to_csv('Revista65.02.csv', index=False)